In [1]:
# Imports
import random

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler

from src.utils import metrics
from src.utils import data_loader as d
from src.utils import models
from src.utils import sliding_window as s

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Load Data
islas_dfs = d.local_data_loader(verbose=False)
# Data transform

results = {}

windows = np.arange(5,20)
islas = ['Tenerife', 'Gran Canaria', 'Lanzarote', 'Fuerteventura', 'La Palma', 'La Gomera', 'El Hierro']

for isla in islas:
    metricas = []
    for window in windows:

        print(f'ISLA: {isla}, WINDOW = {window}')

        # Extract demand value
        demand = islas_dfs[isla]['OBS_VALUE'].values.reshape(-1,1) # data['energy'].values.reshape(-1, 1)

        # Training and validation
        train_size = 1460  # First 4 years for training/validation
        val_size = int(0.2 * train_size)  # 20% of training data for validation
        window_size = window 

        # Split into training, validation, and test sets
        train_data = demand[:train_size - val_size]
        val_data = demand[train_size - val_size:train_size]
        test_data = demand[train_size - window_size:]

        # Sliding window sequences for training and validation
        train_sequences, train_targets = s.create_sequences(train_data, window_size)
        val_sequences, val_targets = s.create_sequences(val_data, window_size)

        # Convert to PyTorch tensors
        train_sequences = torch.tensor(train_sequences, dtype=torch.float32)
        train_targets = torch.tensor(train_targets, dtype=torch.float32)
        val_sequences = torch.tensor(val_sequences, dtype=torch.float32)
        val_targets = torch.tensor(val_targets, dtype=torch.float32)
        # Model training and validation
        from nbeats_pytorch.model import NBeatsNet 

        model = NBeatsNet(
            stack_types=("generic", "generic"),
            forecast_length=1,
            backcast_length=window_size,
            hidden_layer_units=128
        )

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

        train_losses, val_losses = [], []
        best_val_loss = float('inf')
        stopping_counter = 0
        early_stopping = True

        num_epochs = 20_000 
        early_stopping_patience = 1_000

        for epoch in range(num_epochs):
            # Training step
            model.train()
            optimizer.zero_grad()
            _, output = model(train_sequences)
            train_loss = criterion(output, train_targets)
            train_loss.backward()
            optimizer.step()

            # Validation step
            model.eval()
            with torch.no_grad():
                _, val_output = model(val_sequences)
                val_loss = criterion(val_output, val_targets)

            train_losses.append(train_loss.item())
            val_losses.append(val_loss.item())
            if val_loss.item() < best_val_loss:
                best_val_loss = val_loss.item()
                stopping_counter = 0
            else:
                stopping_counter += 1

            if early_stopping and stopping_counter >= early_stopping_patience:
                # print("Early stopping triggered!")
                break

            # if (epoch + 1) % 500 == 0:
                # print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}\n')


        # Prediction
        test_sequences, test_targets = s.create_sequences(test_data, window_size)
        test_sequences = torch.tensor(test_sequences, dtype=torch.float32)
        test_targets = torch.tensor(test_targets, dtype=torch.float32)

        # Initialize lists for storing predictions
        online_predictions = []
        actual_values = []

        # Define online optimizer and loss function
        online_optimizer = optim.Adam(model.parameters(), lr=1e-6)
        criterion = nn.MSELoss()

        # ======= 9. Online Prediction and Evaluation =======
        for i in range(len(test_sequences)):
            x = test_sequences[i].unsqueeze(0)

            # Model inference
            model.eval()
            with torch.no_grad():
                pred = model(x)[1]  # Extract the forecast output
            online_predictions.append(pred.item())
            actual_values.append(test_targets[i].item())

            # Online learning step
            model.train()
            online_optimizer.zero_grad()
            pred_online = model(x)[1]  # Extract forecast again for training
            loss_online = criterion(pred_online, test_targets[i].unsqueeze(0))
            loss_online.backward()
            online_optimizer.step()


        metrics_result = metrics.all_metrics(actual_values, online_predictions)
        print("Evaluation Metrics:")
        print(f"R2: {metrics_result['R2']}")
        for metric, value in metrics_result.items():
            print(f"{metric}: {value:.4f}")
        print('\n')

        # Animation
        test_dates = islas_dfs[isla]['TIME_PERIOD_CODE'].values.reshape(-1,1)
        metricas.append(metrics_result['R2'])
        results[isla] = metricas


ISLA: Tenerife, WINDOW = 5
| N-Beats
| --  Stack Generic (#0) (share_weights_in_stack=False)
     | -- GenericBlock(units=128, thetas_dim=4, backcast_length=5, forecast_length=1, share_thetas=False) at @132354132901584
     | -- GenericBlock(units=128, thetas_dim=4, backcast_length=5, forecast_length=1, share_thetas=False) at @132354137605712
     | -- GenericBlock(units=128, thetas_dim=4, backcast_length=5, forecast_length=1, share_thetas=False) at @132353881930448
| --  Stack Generic (#1) (share_weights_in_stack=False)
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=5, forecast_length=1, share_thetas=False) at @132353952340240
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=5, forecast_length=1, share_thetas=False) at @132353881790992
     | -- GenericBlock(units=128, thetas_dim=8, backcast_length=5, forecast_length=1, share_thetas=False) at @132353881788944
Evaluation Metrics:
R2: 0.6381877200105999
MAE: 129.2124
MSE: 62483.3676
RMSE: 249.9667
R2: 0.6

In [2]:
for isla in islas:
    print(f'\n\nBest window for {isla}: {windows[np.argmax(results[isla])]}')



Best window for Tenerife: 9


Best window for Gran Canaria: 11


Best window for Lanzarote: 11


Best window for Fuerteventura: 8


Best window for La Palma: 7


Best window for La Gomera: 6


Best window for El Hierro: 19
